# August: test demo pdk

In [ ]:
import sys

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")

import matplotlib.pyplot as plt
import numpy as np
import sax

from PhotonicsAI.Photon import utils
from PhotonicsAI.Photon.DemoPDK import *

%matplotlib inline

netlist = """
instances:
  C1:
    component: mzi_2x2_heater_tin_cband
    info:
      dx: 435.0
      dy: 294.75
      ports: 2x2
    settings:
      delta_length: 0
      length: 200
  C2:
    component: mzi_2x2_heater_tin_cband
    info:
      dx: 435.0
      dy: 294.75
      ports: 2x2
    settings:
      delta_length: 0
      length: 100
  C3:
    component: mzi_2x2_heater_tin_cband
    info:
      dx: 435.0
      dy: 294.75
      ports: 2x2
    settings:
      delta_length: 0
      length: 100
placements:
  C1:
    x: 100.0
    y: 498.097
  C2:
    x: 746.111
    y: 749.486
  C3:
    x: 746.111
    y: 248.097
ports:
  o1: C1,o2
  o2: C1,o1
  o3: C2,o1
  o4: C2,o3
  o5: C2,o4
  o6: C3,o2
  o7: C3,o3
  o8: C3,o4
routes:
  optical:
    links:
      C1,o3: C2,o2
      C1,o4: C3,o1
"""

netlist = """
instances:
  C1:
    component: mzi_2x2_heater_tin_cband
    info:
      dx: 435.0
      dy: 294.75
      ports: 2x2
    settings:
      delta_length: 0
      length: 200
placements:
  C1:
    x: 100.0
    y: 498.097
ports:
  o1: C1,o1
  o2: C1,o2
  o3: C1,o3
  o4: C1,o4
"""


d = {}
d["netlist5"] = netlist
d = yaml_netlist_to_gds(d, ignore_links=False)

print(d.keys())


wl = np.linspace(1.5, 1.6, 100)
result = d["sax_circuit"](wl=wl)
d["sax_plot"] = utils.plot_dict_arrays(wl, result)
d["sax_plot"]

In [ ]:
import gdsfactory as gf

from PhotonicsAI.KnowledgeBase.DesignLibrary import (
    _mmi2x2,
    bend_euler,
    heater_tin_cband,
    straight,
)

c = gf.Component()
length = 320
delta_length = 200
xs_1550 = gf.cross_section.cross_section(width=0.5, offset=0, layer="WG")
mmi2x2 = _mmi2x2._mmi2x2()
ref = c << gf.components.mzi2x2_2x2(
    delta_length=delta_length,
    length_y=2.5,
    length_x=length,
    straight_x_top=heater_tin_cband.heater_tin_cband,
    straight_x_bot=heater_tin_cband.heater_tin_cband,
    mirror_bot=True,
    splitter=mmi2x2,
    combiner=mmi2x2,
    cross_section=xs_1550,
)

c.add_port("o1", port=ref.ports["o1"])
c.add_port("o2", port=ref.ports["o2"])
c.add_port("o3", port=ref.ports["o3"])
c.add_port("o4", port=ref.ports["o4"])


def get_model(model="fdtd"):
    m1 = _mmi2x2.get_model(model=model)
    m2 = straight.get_model(model=model)
    m3 = bend_euler.get_model(model=model)
    m4 = heater_tin_cband.get_model(model=model)
    combined_dict = m1 | m2 | m3 | m4
    return combined_dict


recnet = sax.RecursiveNetlist.model_validate(c.get_netlist(recursive=True))
print("Required Models ==>", sax.get_required_circuit_models(recnet))

# print(recnet)
c.plot()
# plt.show()

_c, info = sax.circuit(recnet, get_model())
# print( _c(wl = 1.55) )

plt.figure()
wl = np.linspace(1.53, 1.56, 400)
S = _c(wl=wl)["o1", "o3"]
SS = _c(wl=wl)["o1", "o4"]
plt.plot(wl, np.abs(S) ** 2)
plt.plot(wl, np.abs(SS) ** 2)
plt.show()